In [1]:
import warnings as _warnings
import os as _os
from tempfile import mkdtemp
import numpy as np

class TemporaryDirectory(object):
    """Create and return a temporary directory.  This has the same
    behavior as mkdtemp but can be used as a context manager.  For
    example:

        with TemporaryDirectory() as tmpdir:
            ...

    Upon exiting the context, the directory and everything contained
    in it are removed.
    """

    def __init__(self, suffix="", prefix="tmp", dir=None):
        self._closed = False
        self.name = None  # Handle mkdtemp raising an exception
        self.name = mkdtemp(suffix, prefix, dir)

    def __repr__(self):
        return "<{} {!r}>".format(self.__class__.__name__, self.name)

    def __enter__(self):
        return self.name

    def cleanup(self, _warn=False):
        if self.name and not self._closed:
            try:
                self._rmtree(self.name)
            except (TypeError, AttributeError) as ex:
                # Issue #10188: Emit a warning on stderr
                # if the directory could not be cleaned
                # up due to missing globals
                if "None" not in str(ex):
                    raise
                print("ERROR: {!r} while cleaning up {!r}".format(ex, self,),
                      file=_sys.stderr)
                return
            self._closed = True
            if _warn:
                self._warn("Implicitly cleaning up {!r}".format(self),
                           ResourceWarning)

    def __exit__(self, exc, value, tb):
        self.cleanup()

    def __del__(self):
        # Issue a ResourceWarning if implicit cleanup needed
        self.cleanup(_warn=True)

    # XXX (ncoghlan): The following code attempts to make
    # this class tolerant of the module nulling out process
    # that happens during CPython interpreter shutdown
    # Alas, it doesn't actually manage it. See issue #10188
    _listdir = staticmethod(_os.listdir)
    _path_join = staticmethod(_os.path.join)
    _isdir = staticmethod(_os.path.isdir)
    _islink = staticmethod(_os.path.islink)
    _remove = staticmethod(_os.remove)
    _rmdir = staticmethod(_os.rmdir)
    _warn = _warnings.warn

    def _rmtree(self, path):
        # Essentially a stripped down version of shutil.rmtree.  We can't
        # use globals because they may be None'ed out at shutdown.
        for name in self._listdir(path):
            fullname = self._path_join(path, name)
            try:
                isdir = self._isdir(fullname) and not self._islink(fullname)
            except OSError:
                isdir = False
            if isdir:
                self._rmtree(fullname)
            else:
                try:
                    self._remove(fullname)
                except OSError:
                    pass
        try:
            self._rmdir(path)
        except OSError:
            pass

In [4]:
from math import pi
#from pyfits import writeto,getdata
import sys
import subprocess
import multiprocessing as mp
import os
import shutil
import time
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u
from astropy import constants as const
from astropy.convolution import convolve,convolve_fft,Gaussian2DKernel


def progressBar(name, value, endvalue, bar_length = 25, width = 20):
    
        percent = float(value) / endvalue
        arrow = '-' * int(round(percent*bar_length) - 1) + '>'
        spaces = ' ' * (bar_length - len(arrow))
        sys.stdout.write("\r{0: <{1}} : [{2}]{3}%".format(name, width, arrow + spaces, int(round(percent*100))))
        sys.stdout.flush()
        if value == endvalue:        
             sys.stdout.write('\n\n')

In [5]:
# Model Parameters

deadzone = 50
save_stars = 'n'    #save the stars.dat file?
raypix=0.4           #corresponds to rays per unlensed pixel in the IPM program
ny=1000          #pixels of the magnification map
yl=10               #half size of magnification map in einstein radii
eps=0.02           #maximum flux-loss fraction
halonumber = 20
# ranges and bins of the linear histogram
linran1 = 0
linran2 = 100
linbin = 2000
# ranges and bins of the magnitude histogram
magran1 = -10
magran2 = 10
magbin = 2000

mbh = 30

convolve_radii = [5,100,200] #radii in lightdays
convolve_radii = np.array(convolve_radii)
convolve_radii = (convolve_radii/365*u.lyr).to(u.pc) #convert to parsec


subprocess.call(['gfortran','ipm_treecode.f90','-o', 'program.out','-ffixed-line-length-0','-w'])
fortran_path = "/users/sven/Documents/clustercode"

def einstein_radius(m,z1,z2):
    val = np.sqrt(4*const.G*m*u.Msun*cosmo.angular_diameter_distance_z1z2(z1,z2)/((const.c)**2*cosmo.angular_diameter_distance(z1)*cosmo.angular_diameter_distance(z2)))
    dimlessval = val.to(u.m/u.m)
    return dimlessval.value

def einstein_radius_mpc(m,z1,z2):
    return cosmo.angular_diameter_distance(z2)*einstein_radius(m,z1,z2)

def einstein_radius_lensplane(m,z1,z2):
    return cosmo.angular_diameter_distance(z1)*einstein_radius(m,z1,z2)

def pattern_size_lensplane(yl,m,z1,z2):
    return (2*yl*einstein_radius_lensplane(m,z1,z2)).to(u.pc)

def pixel_size_lensplane(ny,yl,m,z1,z2):
    return (pattern_size_lensplane(yl,m,z1,z2)/ny).to(u.pc)

def pattern_size(yl,m,z1,z2):
    return (2*yl*einstein_radius_mpc(m,z1,z2)).to(u.pc)

def pixel_size(ny,yl,m,z1,z2):
    return (pattern_size(yl,m,z1,z2)/ny).to(u.pc)
    



def run_simulation(counter,qsoname,kappa,gamma,alpha,concentration,radius,z1,z2,blackholemass):
    foldername = qsoname+'/'+str(alpha)+'_'+str(concentration)+'_'+str(radius.value)
    if(counter==0):
        os.mkdir(foldername)
    with TemporaryDirectory(prefix="microlens_tmp_") as tempdir:
        clustermass = blackholemass*concentration
        radius = radius/pixel_size_lensplane(ny,yl,clustermass,z1,z2)
        radius = radius.value
        #randsed=float( input('Please enter a random seed: ')) #random seed input
        ks=kappa*alpha              #convergence in stars
        kc=kappa*(1.-alpha)         #convergence in smooth matter
        ys=2.*yl/(ny-1)             #size of pixel in image plane
        f1=1./abs(1.-kappa-gamma)
        f2=1./abs(1.-kappa+gamma)
        sqrpix=np.sqrt(raypix)
        fmax=max(f1,f2)
        xl1,xl2=1.5*yl*f1, 1.5*yl*f2        #half size of shooting region
        xl=1.5*yl*fmax
        nsmin=3*ks**2/eps/abs((1.-kappa)**2-gamma**2)    #min number of stars
        xmin=np.sqrt(pi*nsmin/ks)/2                     #min half side of stars region
        xls=xl+xmin                                     #account for shooting region
        xnl=abs(ks*(2*xls)*(2*xls)/pi)
        nl=int(xnl)                                     #number of microlenses
        nx1=np.int32(np.round(1.5*ny*f1*sqrpix))
        nx2=np.int32(np.round(1.5*ny*f2*sqrpix))
        nx=max(nx1,nx2)
        thmag=1./(1-kappa-gamma)/(1-kappa+gamma)

        #write the param.dat
        with open(os.path.join(tempdir,'param.dat'),'w') as fil:
            fil.write(str(yl)+' '+str(ny)+'\n')
            fil.write(str(xl)+' '+str(nx)+'\n')
            fil.write(str(ks)+' '+str(kc)+' '+str(gamma)+'\n')
            fil.write(str(1))
            fil.flush()
        #write the stars.dat and the testhalo.dat

        clusternumber=int(nl) #Number of Microlenses
        x1h=np.random.uniform(-xls,xls,clusternumber)
        x2h=np.random.uniform(-xls,xls,clusternumber)
        with open(os.path.join(tempdir,'stars.dat'),'w') as fil:
            for i in range(clusternumber):
                fil.write(str(x1h[i])+' '+str(x2h[i])+' '+str(1)+'\n')
            fil.flush()

        starnumber = halonumber*concentration
        x1s = np.random.normal(0,radius,starnumber)
        x2s = np.random.normal(0,radius,starnumber)
        with open(os.path.join(tempdir,'testhalo.dat'),'w') as fil:
            for i in range(starnumber):
                fil.write(str(x1s[i])+' '+str(x2s[i])+'\n')
            fil.flush()
            
        command = [
            "nice", os.path.join(fortran_path,"program.out"), "stars.dat", "pattern.dat", "param.dat", str(halonumber), str(concentration)]
        proc = subprocess.Popen(
                command, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                cwd=tempdir)
        outs, errs = proc.communicate()
        if proc.returncode != 0:
            print (proc.returncode)
            if True:
                outdir = os.path.join('crashes', "nicaea_data_last_crash")
                if os.path.exists(outdir):
                    shutil.rmtree(outdir)
                shutil.copytree(tempdir, outdir)
            if True:
                print(">>> stdout <<<\n", outs.decode("utf-8"))
                print(">>> stderr <<<\n", errs.decode("utf-8"))
                raise subprocess.CalledProcessError(
                    proc.returncode, " ".join(command))
            else:
                return None
            
        result = createfits(os.path.join(tempdir,"pattern.dat"),ny)
        convradii = ((convolve_radii/pixel_size(ny,yl,clustermass,z1,z2)).to(u.pc/u.pc)).value
#         print(convradii)
        fitsmap1 = result #convolve_fft(result,Gaussian2DKernel(convradii[1]))
        fitsmap1 = fitsmap1+0.0000001
        fitsmap2 = convolve_fft(result,Gaussian2DKernel(convradii[2]))
        fitsmap2 = fitsmap2+0.0000001
        fitsmap3 = fitsmap1/fitsmap2
        fitsmaps = [fitsmap1,fitsmap2,fitsmap3]
        for ii in range(3):
#             print("Creating Histogram", ii)
            if(np.isnan(fitsmap1).any() or fitsmap1.any() <= 0.000001):
                print("Error")
                sys.exit()
            if(np.isnan(fitsmap2).any() or fitsmap2.any() <= 0.000001):
                print("Error2")
                sys.exit()
            if(np.isnan(fitsmap3).any() or fitsmap3.any() <= 0.000001):
                print("Error3")
                sys.exit()
            savename = foldername+'/'+str(ii)+'_'+str(counter)
            create_histogram(savename,fitsmaps[ii],thmag)
        return fitsmap1

        


def create_histogram(savename,fitsmap,thmag):
#     fil = open(savename+'log','w')
#     for i in range(ny):
#         for j in range(ny):
#             fil.write(str(fitsmap[i,j])+' ')
#         fil.write('\n')
#     fil.close()
    fitsmap = fitsmap[deadzone:ny-deadzone,deadzone:ny-deadzone]/thmag
#     plt.imshow(fitsmap)
#     plt.colorbar()
#     plt.show()
    fitsmap = np.ravel(fitsmap)
    m = np.mean(fitsmap)
#     print('creating linhist')
    linhist,linedges = np.histogram(fitsmap, bins=linbin, range = (linran1,linran2))
#     print('taking log')
    fitsmap = -2.5*np.log10(fitsmap)
#     print('creating maghist')
    maghist,magedges = np.histogram(fitsmap, bins=magbin, range = (magran1,magran2))    
    np.save(savename,[linhist,linedges,maghist,magedges])
    
def createfits(filein,size):
    size=int(size)
    a=np.zeros((size,size))	  # Create a numpy square array of the right size
    n=0			  # Initialize counter
    if (os.path.exists(filein)):		# Check that input file exists
        for line in open(filein, 'r'):	# Loop over lines
            i=int(n%size)			# This is row number
            j=int(n/size)			# This is column number
            a[j,i]=float(line)		# Store data on array
            n+=1
        return a
    else:			  # If input file does not exist exit
        print ("File",filein, "does not exist")


def simulate_quasar(nsims,nprocs,alphas,concs,radii,qsoname,z1,z2,kappa,gamma):
    exitflag = True
    runcounter = 0
    arglist = []
    if not os.path.exists(qsoname):
        os.mkdir(qsoname)
    for alpha in alphas:
        for concentration in concs:
            for radius in radii:
                for counter in range(nsims):
                    arglist.append([counter,qsoname,kappa,gamma,alpha,concentration,radius,z1,z2,mbh])
    length = len(arglist)
    while(exitflag):
        progressBar(qsoname,runcounter,length)
        for i in range(nprocs):
            if(runcounter+i<length):
                job = mp.Process(target=run_simulation, args=arglist[runcounter+i])
                job.start()
            else:
                exitflag=False
        for i in range(nprocs):
            if(runcounter+i<length):
                job.join()
                job.terminate()
        runcounter = runcounter+nprocs

In [8]:
nsims = 20
nprocs = 100

data = open('lenses-milib_A.dat','r')
lenses = data.readlines()
data.close()
nl=0
for lens in lenses:
    [name,kappa,gamma,ds,dl,dls,temp0,temp1] = lens.split()
    kappa = float(kappa)
    gamma = float(gamma)
    nl=nl+1
    
lens = lenses[0]
[qsoname,kappa,gamma,ds,dl,dls,temp0,temp1] = lens.split()
z1 = 0.4
z2 = 0.9
kappa = float(kappa)
gamma = float(gamma)
qsoname = qsoname.replace(" ","")
alphas = [0.1,0.2,0.5,0.8]
radii = [0.2]*u.pc
concs = [300]

import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    simulate_quasar(nsims,nprocs,alphas,concs,radii,qsoname,z1,z2,kappa,gamma)



rxj1131a             : [>                        ]0%

In [172]:
print(1./(1-kappa-gamma)/(1-kappa+gamma))


3.703703703703704
